# Notebook for training information retrieval models

### Import packages

In [1]:
# Import packages
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SQLContext
from pyspark.sql.functions import udf, size, explode, col, countDistinct, collect_list, monotonically_increasing_id, row_number
from pyspark.ml import Pipeline

from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS as gensim_words
import spacy
sp = spacy.load('en_core_web_sm')

import os

from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner

from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Normalizer as Normalizer_L2

from pyspark.sql.types import StructField, StructType, IntegerType, FloatType

In [2]:
nltk_stopwords = set(stopwords.words('english')) \
                    .union(set(stopwords.words('german'))) \
                    .union(set(stopwords.words('french')))
gensim_stopwords = set(gensim_words)
spacy_stopwords = sp.Defaults.stop_words
# https://countwordsfree.com/stopwords
cwf_stopwords = set(line.strip() for line in open('stop_words.txt'))

all_stopwords = list( nltk_stopwords \
                        .union(gensim_stopwords) \
                        .union(spacy_stopwords) \
                        .union(cwf_stopwords) )

### Create Spark Context and SQL Context

In [3]:
# Get the right paths on local machine
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["PYSPARK_PYTHON"] = '/usr/bin/python3.7'
os.environ["PYSPARK_DRIVER_PYTHON"] = '/usr/bin/python3.7'

In [4]:
# Start spark session configured for spark nlp
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('SDDM') \
        .config('spark.driver.memory', '8g') \
        .config('spark.executor.memory', '8g') \
        .config('spark.memory.fraction', '0.8') \
        .config('spark.executor.cores', '8') \
        .config('spark.local.dir', '/home/rikz/Documents/Master/Semester2/SDDM/data/tmp') \
        .config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.0') \
        .getOrCreate()
print("Created a SparkSession")
sc = spark.sparkContext
print("Created a SparkContext")
sqlContext = SQLContext(sc)
print("Created a SQLContext")

#         .config('spark.local.dir', '/data/s1847503/SDDM/tmp') \

Created a SparkSession
Created a SparkContext
Created a SQLContext


### Load the data into a SQLContext Dataframe

In [5]:
df = sqlContext.read.format('csv').options(header='true', maxColumns=2000000) \
        .load('/home/rikz/Documents/Master/Semester2/SDDM/data/data.csv')
#       .load('/data/s1847503/SDDM/newdata/data.csv')
df.show()

+---+--------------------+--------------------+--------------------+--------------------+
|_c0|            paper_id|               title|        list_authors|           full_text|
+---+--------------------+--------------------+--------------------+--------------------+
|  0|           question0|                   -|                   -|How does temperat...|
|  1|           question1|                   -|                   -|Seasonality of tr...|
|  2|           question2|                   -|                   -|Effectiveness of ...|
|  3|           question3|                   -|                   -|Effectiveness of ...|
|  4|           question4|                   -|                   -|Effectiveness of ...|
|  5|           question5|                   -|                   -|Effectiveness of ...|
|  6|           question6|                   -|                   -|Effectiveness of ...|
|  7|           question7|                   -|                   -|Effectiveness of ...|
|  8|     

### Initialize Annotators

In [6]:
# Pipeline for text
document_assembler = DocumentAssembler() \
                        .setInputCol('full_text') \
                        .setOutputCol('document')

# Tokenizer divides the text into tokens
tokenizer = Tokenizer() \
                .setInputCols(['document']) \
                .setOutputCol('tokens')

# Finisher converts tokens to human-readable output (we need the tokens for determining the text lengths)
finisher_tokens = Finisher() \
                        .setInputCols(['tokens']) \
                        .setCleanAnnotations(False)

# Normalizer removes punctuation, numbers etc.
normalizer = Normalizer() \
                .setInputCols(['tokens']) \
                .setOutputCol('normalized') \
                .setLowercase(True)

# Lemmatizer changes each word to its lemma
lemmatizer = LemmatizerModel.pretrained() \
                .setInputCols(['normalized']) \
                .setOutputCol('lemma')

# StopWordsCleaner removes stop words    
stopwords_cleaner = StopWordsCleaner() \
                        .setInputCols(['lemma']) \
                        .setOutputCol('clean_lemma') \
                        .setCaseSensitive(False).setStopWords(all_stopwords)

# Finisher converts clean tokens to human-readable output
finisher = Finisher() \
            .setInputCols(['clean_lemma']) \
            .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


### Create Pipeline

In [7]:
# Pipeline for fully preprocessing the text
pipeline = Pipeline() \
            .setStages([
                document_assembler,
                tokenizer,
                normalizer,
                lemmatizer,
                stopwords_cleaner,
                finisher_tokens,
                finisher
             ])

### Preprocess questions

In [8]:
# questions = sqlContext.read.format('csv').options(header='true').load('/data/s1847503/SDDM/newdata/questions.csv')
questions = sqlContext.read.format('csv').options(header='true').load('/home/rikz/Documents/Master/Semester2/SDDM/data/questions.csv')
questions_clean = pipeline.fit(questions).transform(questions)
questions_clean = questions_clean.select('question_id', 'full_text', col('finished_clean_lemma').alias('preprocessed'))
questions_clean.show()

+-----------+--------------------+--------------------+
|question_id|           full_text|        preprocessed|
+-----------+--------------------+--------------------+
|          0|How does temperat...|[temperature, hum...|
|          1|Seasonality of tr...|[seasonality, tra...|
|          2|Effectiveness of ...|[effectiveness, i...|
|          3|Effectiveness of ...|[effectiveness, p...|
|          4|Effectiveness of ...|[effectiveness, s...|
|          5|Effectiveness of ...|[effectiveness, c...|
|          6|Effectiveness of ...|[effectiveness, m...|
|          7|Effectiveness of ...|[effectiveness, c...|
|          8|Significant chang...|[change, transmis...|
|          9|Effectiveness of ...|[effectiveness, w...|
+-----------+--------------------+--------------------+



In [9]:
# Select the question from 0 to 9
question_num = 4

questions_clean = questions_clean.filter(questions_clean.question_id == question_num)
questions_clean.first().full_text

'Effectiveness of school distancing'

### Preprocess text

In [10]:
# Peprocess the data
df = pipeline.fit(df).transform(df)
df = df.select('*', size('finished_tokens').alias('text_length'))

# Keep only papers with a text length of greater than 10
print("Before removing too short papers and duplicates: {} rows.".format(df.count()))
df = df.dropna(subset='full_text')
df = df.filter(df.text_length > 10)
print("Removed very short papers")
df = df.dropDuplicates(subset=['full_text'])
print("Removed duplicates")
print("After removing too short papers and duplicates: {} rows.".format(df.count()))
print()

df = df.select(
                col('_c0').alias('id'),
                'paper_id',
                'title',
                'full_text',
                'text_length',
                col('finished_clean_lemma').alias('preprocessed')
            )

df.show()

Before removing too short papers and duplicates: 818 rows.
Removed very short papers
Removed duplicates
After removing too short papers and duplicates: 807 rows.

+---+--------------------+--------------------+--------------------+-----------+--------------------+
| id|            paper_id|               title|           full_text|text_length|        preprocessed|
+---+--------------------+--------------------+--------------------+-----------+--------------------+
|504|44f5bf79fa41bef5e...|Nanomolecular Dia...|"Clinical applica...|        388|[clinical, applic...|
|798|5a5aba543db0ffbf3...|Novel picornaviru...|"The family Picor...|        262|[family, picornav...|
|394|3f8d3e90029049640...|Protective effect...|Hypercholesterole...|       3012|[hypercholesterol...|
|571|6ac37ecc83bd4e2f6...|ARTICLE Molecular...|Total joint arthr...|       5483|[total, joint, ar...|
| 73|ee2532faa109395ff...|                null|"Imagine that we ...|        404|[imagine, operate...|
|736|9cc3d3f648719e37

### TF-IDF

In [11]:
# Compute TF-IDF matrix for papers
tf_p = []
tf_idf_papers = []

tf_p = HashingTF(inputCol='preprocessed', outputCol='tf') \
                    .transform(df)

tf_idf_papers = IDF(inputCol='tf', outputCol='feature') \
                        .fit(tf_p) \
                        .transform(tf_p)

In [12]:
# Compute TF-IDF matrix for questions
tf_q = []
tf_idf_questions = []

tf_q = HashingTF(inputCol='preprocessed', outputCol='tf') \
                    .transform(questions_clean)

tf_idf_questions = IDF(inputCol='tf', outputCol='feature') \
                        .fit(tf_p) \
                        .transform(tf_q)

In [13]:
# tf_idf_papers.show()
tf_idf_questions = tf_idf_questions.select('question_id', 'feature')
tf_idf_papers = tf_idf_papers.select('id', 'feature')

In [14]:
# Compute L2-norm for papers and questions
normalizer_L2 = Normalizer_L2(inputCol='feature', outputCol='norm')
tf_idf_papers = normalizer_L2.transform(tf_idf_papers)
tf_idf_questions = normalizer_L2.transform(tf_idf_questions)

In [15]:
# Compute similarity matrix
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
matrix_q = IndexedRowMatrix(
                tf_idf_questions \
                    .select('question_id', 'norm') \
                    .rdd.map(lambda row: IndexedRow(row.question_id, row.norm.toArray()))
            ).toBlockMatrix()

matrix_p = IndexedRowMatrix(
                tf_idf_papers \
                    .select('id', 'norm') \
                    .rdd.map(lambda row: IndexedRow(row.id, row.norm.toArray()))
            ).toBlockMatrix()

In [16]:
sim_matrix = matrix_p.multiply(matrix_q.transpose())

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60466)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/usr/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.pro

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35227)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35227)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35227)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35227)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most

Py4JError: An error occurred while calling o958.multiply

In [ ]:
sim_matrix = sim_matrix.toLocalMatrix().toArray()

In [ ]:
relevant = sc.parallelize(sim_matrix[:, question_num].tolist()) \
                .zipWithIndex() \
                .toDF(['similarity', 'paper_id'])

# Remove questions from the paper list
# Sort on cosine similarity
# Take the top 10 relevant documents
relevant = relevant.select('paper_id', 'similarity') \
                .filter(relevant.paper_id > 9) \
                .sort(col('similarity').desc()) \
                .limit(10)

relevant.show()

In [ ]:
relevant_ids = [int(row.paper_id) for row in relevant.collect()]

count = 1
for r in relevant_ids:
    print("{}: {}".format(count, df.filter(df.id == r).first().title))
    print()
    count += 1

### Word2Vec

In [ ]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(inputCol='preprocessed', outputCol='word_vector')

In [ ]:
model = word2Vec.fit(df)
df = model.transform(df)

In [ ]:
df.show()

In [ ]:
questions_clean = model.transform(questions_clean)
ques_vec = questions_clean.first().word_vector

In [ ]:
# Calculate cosine similarity between a document vector and a question vector
def cossim(doc_vec): 
    global ques_vec
    sim = np.dot(doc_vec, ques_vec) / np.sqrt(np.dot(doc_vec, ques_vec)) / np.sqrt(np.dot(doc_vec, ques_vec)) 
    return float(sim)

cossim_udf = udf(cossim)

In [ ]:
df2 = df.select('id', cossim_udf('word_vector').alias('similarity'))

In [ ]:
df2.show()

In [ ]:
# print("Before removing empty papers: {} rows.".format(df2.count()))
print("x")
df2 = df2.filter(df2.similarity.isNotNull())
print("After removing empty papers 1: {} rows.".format(df2.count()))
df2 = df2.filter(df2.similarity != 'NaN')
# df2 = df2.dropna(subset='similarity')
print("After removing empty papers 2: {} rows.".format(df2.count()))
df2 = df2.orderBy('similarity', ascending=False).show()

In [ ]:
relevant = [40821, 40931, 40823, 40776, 40831]
for r in relevant:
    print(df.filter(df.id == 40821).first().title)
    print()

### Close Spark Context when done

In [17]:
sc.stop()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35227)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
/usr/local/lib/python3.7/dist-packages/pyspark/context.py:446: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning
